In [1]:
import pandas as pd
import saspy
from IPython.display import HTML
from us import states
from census import Census

First, connect to the census using our API key

In [2]:
c = Census("e7cd51b9bf8678c6e1c9944116f0984b44cded26")

Next, download a couple of census questions.   We're going to look at the ACS 5-year estimates here; just for a starting point let's get the male and female population of the states.  B01001_002E is male, and B0101_026E is female.  You can find the full list of variables [in the US Census API documentation](https://api.census.gov/data/2015/acs/acs5/variables.html).

In [3]:
cen_data = pd.DataFrame(c.acs5.state(('NAME','B01001_002E','B01001_026E'), Census.ALL))

Now, we need to upload this to a SAS dataset, since we're SAS programmers!  Here's a simple function that does that:

In [4]:
def upload_to_SAS(df,path,sastbl):
    sas = saspy.SASsession(cfgname='winlocal')
    sas.saslib('demo',path=path)
    sas.df2sd(df=df,table=sastbl,libref='demo')

This takes three arguments, a dataframe, a path to save it to, and the name of the SAS dataset to call it.

Now that we've invoked it, we can upload it to SAS!

In [5]:
upload_to_SAS(df=cen_data,path='c:\\temp',sastbl='uscen_data')

SAS Connection established. Subprocess id is 2468

5                                                          The SAS System                             14:13 Tuesday, October 1, 2019

31         
32         libname demo    'c:\temp'  ;
NOTE: Libref DEMO was successfully assigned as follows: 
      Engine:        V9 
      Physical Name: c:\temp
33         
34         


Now we're cooking.  We have a SAS dataset.  Now, before we hand off to SAS entirely, let's make sure what we have is okay.

First, we have to create a new SAS connection.

In [6]:
sas =saspy.SASsession(cfgname='winlocal')

SAS Connection established. Subprocess id is 3140



This creates the SAS session, basically telling SAS to spawn a sas.exe process, and it tells it to use the 'winlocal' (windows local) configuration.  If you have a SAS Server environment, you might have a different value here.

In [7]:
sas.saslib('demo',path='c:\\temp')

5                                                          The SAS System                             14:13 Tuesday, October 1, 2019

31         
32         libname demo    'c:\temp'  ;
NOTE: Libref DEMO was successfully assigned as follows: 
      Engine:        V9 
      Physical Name: c:\temp
33         
34         


This tells SAS to create a libname.

In [8]:
cd = sas.sasdata('uscen_data','demo')

Here, we're creating a connection to a specific SAS dataset.  Now, any reference to `cd` will refer to our dataset.

Now, let's get some information out of it.

In [9]:
cd.columnInfo()

SAS Connection terminated. Subprocess id was 2468


,Member,Num,Variable,Type,Len,Pos
0,DEMO.USCEN_DATA,1,B01001_002E,Num,8,0
1,DEMO.USCEN_DATA,2,B01001_026E,Num,8,8
2,DEMO.USCEN_DATA,3,NAME,Char,20,16
3,DEMO.USCEN_DATA,4,state,Char,2,36


This will give us basically a PROC CONTENTS of the dataset.

In [10]:
cd.head()

,B01001_002E,B01001_026E,NAME,state
0,2346193,2494971,Alabama,01
1,385296,351559,Alaska,02
2,3344106,3384471,Arizona,04
3,1456694,1511778,Arkansas,05
4,19200970,19453236,California,06


`head` is telling it to give us the top several rows, just so we can make sure it makes sense.

One thing to note: Python is 0-based, not 1-based, so don't get confused if you run the same thing inside SAS and see the row numbers don't line up!

Okay, we have sensible population numbers.  But our labels are missing, and our variable names make no sense, so let's clean those up. Easiest way is to submit some SAS code directly, so let's do that, with `submit()`.

In [11]:
rc = sas.submit("""
proc datasets lib=demo;
modify uscen_data;
rename b01001_002E = male
       b01001_026E = female;
run;
label male="Population of males in state"
      female="Population of females in state";
run; quit;

""")
rc['LOG']

'\x0c19                                                         The SAS System                             14:13 Tuesday, October 1, 2019\n\n145        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode=\'inline\') device=svg style=HTMLBlue;\n145      ! ods graphics on / outputfmt=png;\nNOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1\n146        \n147        \n148        proc datasets lib=demo;\n149        modify uscen_data;\n150        rename b01001_002E = male\n151               b01001_026E = female;\nNOTE: Renaming variable b01001_002E to male.\nNOTE: Renaming variable b01001_026E to female.\n152        run;\n\nNOTE: MODIFY was successful for DEMO.USCEN_DATA.DATA.\n153        label male="Population of males in state"\n154              female="Population of females in state";\n155        run;\n\n155      !      quit;\n\nNOTE: PROCEDURE DATASETS used (Total process time):\n      real time           0.17 seconds\n      cpu time            0.04 sec

Now, let's check if that worked properly.

In [12]:
cd.head()

,male,female,NAME,state
0,2346193,2494971,Alabama,01
1,385296,351559,Alaska,02
2,3344106,3384471,Arizona,04
3,1456694,1511778,Arkansas,05
4,19200970,19453236,California,06


Okay, so we have a dataset with decent variable names.  Now let's turn it over to SAS.